# Importations

In [7]:
import sys
MOD_DIR = '/home/16012858/game-learning'

if MOD_DIR not in sys.path:
    sys.path.append(MOD_DIR)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb

from sklearn import datasets
from sklearn import svm, naive_bayes
from sklearn import tree
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import cross_val_score
from mod.wine_regression import *

## Directory

In [10]:
INPUT_DIR = f"{MOD_DIR}/data/Sep-09-2022_10000matches.csv"

# Importation des données et pre-processing

In [15]:
data = pd.read_csv(INPUT_DIR1)
data['win'] = np.where(data['win'] == True, 1, 0)
data.head(20)

/tmp/ipykernel_160/1505835559.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(INPUT_DIR1)


,Unnamed: 0,assists,GameID,GameMode,baronKills,bountyLevel,champExperience,champLevel,championName,damageDealtToObjectives,...,totalTimeSpentDead,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,0,20,NA1_4430908515,ARAM,0,0,14531,15,Ashe,112,...,74,797,465,1,4,5,0,0,0,0
1,0,3,4035079604,CLASSIC,0,3,15453,16,Sett,18147,...,156,2556,686,4,4,10,0,0,7,1
2,0,19,4035079604,CLASSIC,0,3,13168,15,Trundle,27626,...,82,956,1571,6,4,21,3,0,11,1
3,0,6,4035079604,CLASSIC,0,1,13619,15,Sylas,10629,...,127,600,935,6,4,8,0,0,5,1
4,0,7,4035079604,CLASSIC,1,1,10674,13,KogMaw,13624,...,118,2202,521,3,4,3,1,1,2,1
5,0,24,4035079604,CLASSIC,0,0,11295,13,Nami,2707,...,21,1284,352,5,4,68,5,7,31,1
6,0,1,4035079604,CLASSIC,0,0,11591,14,Yone,8203,...,329,1670,1268,0,10,4,0,1,1,0
7,0,5,4035079604,CLASSIC,0,0,10064,13,Viego,6896,...,278,84,1725,2,10,17,3,2,3,0
8,0,2,4035079604,CLASSIC,0,0,13593,15,Yasuo,11150,...,196,1266,1560,4,10,12,0,1,8,0
9,0,5,4035079604,CLASSIC,0,0,8732,12,Caitlyn,757,...,206,652,1713,0,10,9,1,0,7,0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100001 entries, 0 to 100000
Data columns (total 69 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   Unnamed: 0                      100001 non-null  int64 
 1   assists                         100001 non-null  int64 
 2   GameID                          100001 non-null  object
 3   GameMode                        100001 non-null  object
 4   baronKills                      100001 non-null  int64 
 5   bountyLevel                     100001 non-null  int64 
 6   champExperience                 100001 non-null  int64 
 7   champLevel                      100001 non-null  int64 
 8   championName                    100001 non-null  object
 9   damageDealtToObjectives         100001 non-null  int64 
 10  damageSelfMitigated             100001 non-null  int64 
 11  deaths                          100001 non-null  int64 
 12  dragonKills                   

In [17]:
data.describe()

,Unnamed: 0,assists,baronKills,bountyLevel,champExperience,champLevel,damageDealtToObjectives,damageSelfMitigated,deaths,dragonKills,...,totalTimeSpentDead,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
count,100001.0,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,...,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000,100001.000000
mean,0.0,8.183938,0.091579,1.003130,12794.911191,13.991690,9776.702483,17229.739303,6.201018,0.406046,...,184.968840,1270.381536,1312.757882,2.028500,5.587534,23.859081,1.796792,2.434006,10.272437,0.499995
std,0.0,5.776155,0.318392,2.075611,4518.977627,2.846682,10103.122411,14951.622587,3.406475,0.837422,...,119.276347,1888.306171,1440.222562,1.971139,3.720909,17.719650,2.226332,2.634249,8.110498,0.500002
min,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,4.000000,0.000000,0.000000,9800.000000,12.000000,2357.000000,7446.000000,4.000000,0.000000,...,97.000000,186.000000,417.000000,0.000000,2.000000,12.000000,0.000000,1.000000,5.000000,0.000000
50%,0.0,7.000000,0.000000,0.000000,12612.000000,14.000000,6368.000000,12694.000000,6.000000,0.000000,...,170.000000,701.000000,879.000000,2.000000,6.000000,19.000000,1.000000,2.000000,8.000000,0.000000
75%,0.0,11.000000,0.000000,1.000000,15610.000000,16.000000,14118.000000,22098.000000,8.000000,0.000000,...,255.000000,1470.000000,1700.000000,3.000000,9.000000,30.000000,3.000000,3.000000,12.000000,1.000000
max,0.0,46.000000,4.000000,29.000000,43518.000000,18.000000,96235.000000,203953.000000,28.000000,6.000000,...,1447.000000,39571.000000,29237.000000,11.000000,11.000000,193.000000,37.000000,34.000000,100.000000,1.000000


In [19]:
#data.drop(['Unnamed: 0', 'GameID', 'championName', 'GameMode'], axis=1)

,assists,baronKills,bountyLevel,champExperience,champLevel,damageDealtToObjectives,damageSelfMitigated,deaths,dragonKills,firstBloodAssist,...,totalTimeSpentDead,trueDamageDealtToChampions,trueDamageTaken,turretTakedowns,turretsLost,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,20,0,0,14531,15,112,3250,3,0,True,...,74,797,465,1,4,5,0,0,0,0
1,3,0,3,15453,16,18147,30659,6,0,False,...,156,2556,686,4,4,10,0,0,7,1
2,19,0,3,13168,15,27626,28297,4,2,True,...,82,956,1571,6,4,21,3,0,11,1
3,6,0,1,13619,15,10629,19937,5,1,True,...,127,600,935,6,4,8,0,0,5,1
4,7,1,1,10674,13,13624,6303,6,0,False,...,118,2202,521,3,4,3,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,5,0,0,10796,13,1137,15053,7,0,False,...,251,528,1134,0,10,16,1,0,5,0
99997,2,0,0,10318,13,6707,11223,12,0,False,...,336,5998,1147,0,10,14,1,0,7,0
99998,4,0,0,9898,12,337,8258,10,0,False,...,227,1077,433,0,10,16,1,0,9,0
99999,1,0,0,8304,11,3931,11403,7,0,False,...,166,272,116,0,10,17,1,3,9,0
